# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing

## 1.1. Download Dataset

In [3]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1vF3FqgBC1Y-RPefeVmY8zetdZG1jmHzT'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_train.csv')

id = '1XhaV8YMuQeSwozQww8PeyiWMJfia13G6'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_test.csv')

import pandas as pd
df_train = pd.read_csv("imdb_train.csv")
df_test = pd.read_csv("imdb_test.csv")

reviews_train = df_train['review'].tolist()
sentiments_train = df_train['sentiment'].tolist()
reviews_test = df_test['review'].tolist()
sentiments_test = df_test['sentiment'].tolist()

print("Training set number:",len(reviews_train))
print("Testing set number:",len(reviews_test))

Training set number: 25000
Testing set number: 25000


## 1.2. Preprocess data

*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *

In [0]:
# Please comment your code
import pprint as pp
import re
from bs4 import BeautifulSoup

#CSV file has 25000 reviews. But these reviews have < br/> tags, which are removed using regex expressions

def remove_HTML_tags(x):
    x = re.sub(r'\<br \/\>',' ',x)   #removes <br \> tags
    removeSlash = re.compile(r"[\\\']")
    #replacement = re.compile(r"\'")
    #x = re.sub(r"[^a-z0-9]+", " ", x.lower()) 
    #x = re.sub(removeSlash,'^',x) 
    return x


reviews_train_cleaned = []

for item in range(0, len(reviews_train)):
  reviews_train_cleaned.append(remove_HTML_tags(reviews_train[item]))



# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

In [0]:
# Please comment your code

# We will be implementing Word2Vec with skipgrams


### 2.1.1. Data Preprocessing for Word Embeddings

*You are required to describe which preprocessing techniques were used with justification of your decision.*

**Important**: If you are going to use the code from lab3 word2vec preprocessing. Please note that `word_list = list(set(word_list)) ` has randomness. So to make sure the word_list is the same every time you run it, you can put `word_list.sort()` after that line of code.

In [5]:
import nltk
nltk.download('punkt')

import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize



import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

#removal of all punctuations - .,?!_-
def remove_punctuation_re(x):
    #x = re.sub(r'[^\w\s]','',x)
    x = re.sub(r'[^\w\s]',' ',x)    #replacing with space, maybe it will help?
    return x

def remove_underscores_re(x):
    x = re.sub(r'_',' ',x)    #replacing with space, maybe it will help?
    return x

def remove_numbers_re(x):
  x = re.sub(r'[0-9]+',' ',x)    #replacing with space, maybe it will help?
  return x


reviews_train_Without_Punctuations = []

# Removing all punctuations
for item in range(0, len(reviews_train_cleaned)):
  reviews_train_Without_Punctuations.append(remove_punctuation_re(reviews_train_cleaned[item]))

reviews_train_Without_Underscore = []

#Removing underscore
for item in range(0, len(reviews_train_Without_Punctuations)):
  reviews_train_Without_Underscore.append(remove_underscores_re(reviews_train_Without_Punctuations[item]))

reviews_train_Without_Numbers = []
#Remove numbers
for item in range(0, len(reviews_train_Without_Underscore)):
  reviews_train_Without_Numbers.append(remove_numbers_re(reviews_train_Without_Underscore[item]))


# Tokenising the sentence to process it by using NLTK library
sent_text_NLTK =[]
for i in range (0, len(reviews_train_Without_Numbers)):
  sent_text_NLTK.append(sent_tokenize(reviews_train_Without_Numbers[i]))

#pp.pprint(sent_text_NLTK[0])



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
nltk.download('stopwords')

from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize


normalized_text = []

#tokenizing the sentences
for reviews in sent_text_NLTK:
  for sent in reviews:
    #tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower())
    tokens = word_tokenize(sent.lower()) #converting all words to lower case 
    #for token in tokens:
    normalized_text.append(tokens)

text_train_ns=[]
stop_words = sw.words()
for tokens in normalized_text:
    filtered_sentence = [w for w in tokens if not w in stop_words]
    text_train_ns.append(filtered_sentence)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
newlist = []
list_unique_words = []
#print(text_train_ns[0])


def remove_accentedCharacters_re(x):
  x = re.sub(r'[^a-z]','',x)    #replacing with space, maybe it will help?
  return x


#Flatening the List of list to a single list - A single list of all words
for sentences in text_train_ns:
  for words in sentences:
    words = words.lower()
    newlist.append(remove_accentedCharacters_re(words))    # remove accented chars


#Assigning the unaltered list
word_sequence = newlist

#Removal of duplicates
set_Unique_Words = set (newlist)

#Converting the set back to a list
for items in set_Unique_Words:
  list_unique_words.append(items)

list_unique_words.sort()

#Assigning the unique word list
word_list = list_unique_words


In [0]:
# make dictionary so that we can reference each index of unique word
word_dict = {w: i for i, w in enumerate(word_list)}

# Making window size 1 skip-gram
# i.e.) he likes cat
#   -> (he, [likes]), (likes,[he, cat]), (cat,[likes])
#   -> (he, likes), (likes, he), (likes, cat), (cat, likes)
skip_grams = []


# Word_Sequence is the unaltered list of words/ tokens from the corpus
for i in range(1, len(word_sequence) - 1):
    # (context, target) : ([target index - 1, target index + 1], target)
    target = word_dict[word_sequence[i]]
    context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]

    # skipgrams - (target, context[0]), (target, context[1])..
    for w in context:
        skip_grams.append([target, w])



In [0]:
voc_size = len(word_list)

# prepare random batch from skip-gram - we do not have enought data so we randomly select data
def prepare_batch(data, size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=False)

    for i in random_index:
        input_temp = [0]*voc_size
        input_temp[data[i][0]] = 1
        random_inputs.append(input_temp)  # target
        random_labels.append(data[i][1])  # context word

    return np.array(random_inputs), np.array(random_labels)


learning_rate = 0.1
batch_size = 800
embedding_size = 51   #max length of characters
no_of_epochs = 100

### 2.1.2. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np



class SkipGram(nn.Module):
    def __init__(self):
        super(SkipGram, self).__init__()
        #You need to use "bias=False" when you define Linear functions
        #***************put your code here***************
        self.linear1 = nn.Linear(voc_size, embedding_size, bias=False)
        self.linear2 = nn.Linear(embedding_size, voc_size, bias=False)

    def forward(self, x):
        hidden = self.linear1(x)
        out = self.linear2(hidden)
        return out


      

### 2.1.3. Train Word Embeddings Model

In [11]:
# Please comment your code
skip_gram_model = SkipGram()
criterion = nn.CrossEntropyLoss() #please note we are using "CrossEntropyLoss" here
optimiser = optim.SGD(skip_gram_model.parameters(), lr=learning_rate)



#outputs = torch.from_numpy(outputs)
for epoch in range(100):

    inputs,labels = prepare_batch(skip_grams, batch_size)
    inputs_torch = torch.from_numpy(inputs).float()  #feeding one hot encodings of context word
    labels_torch = torch.from_numpy(labels)          #feeding label of target word

    #***************put your code here***************
    # 1. zero grad
    # 2. forword propagation
    # 3. calculate loss
    # 4. back propagation

    skip_gram_model.train()
    # zero the parameter gradients
    optimiser.zero_grad()

    # forward + backward + optimize
    
    output_label = skip_gram_model(inputs_torch)
    loss = criterion(output_label, labels_torch) # We don't need to calcualte logsoftmax here
    loss.backward()
    optimiser.step()


    if epoch % 10 == 0: 
      print('Epoch: %d, loss: %.4f' %(epoch + 1, loss))


#get the weight from a Model Linear layer
weight = skip_gram_model.linear2.weight
trained_embeddings = weight.detach().T.numpy()

Epoch: 1, loss: 10.1845
Epoch: 11, loss: 10.1845
Epoch: 21, loss: 10.1846
Epoch: 31, loss: 10.1845
Epoch: 41, loss: 10.1845
Epoch: 51, loss: 10.1846
Epoch: 61, loss: 10.1844
Epoch: 71, loss: 10.1845
Epoch: 81, loss: 10.1844
Epoch: 91, loss: 10.1845


### 2.1.4. Save Word Embeddings Model

In [14]:
# Please comment your code
torch.save(skip_gram_model, 'word_embedding_model.pt')


NameError: ignored

### 2.1.5. Load Word Embeddings Model

In [0]:
# File is saved with sharing mode on, kindly run this cell to download onto your drive
# Run the model class first for this to work 
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import torch
import torch.nn as nn
import torch.optim as optim

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1-05c9pMCI-F3EJ9G7hnw7Y1ibB75rVoK'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('word_embedding_model.pt')



In [0]:
saved_skip_gram_model = torch.load('word_embedding_model.pt')
saved_skip_gram_model.eval()
skipgram_weight = saved_skip_gram_model.linear2.weight
trained_embeddings_skipgram = skipgram_weight.detach().T.numpy()

## 2.2. Character Embeddings

### 2.2.1. Data Preprocessing for Character Embeddings

*You are required to describe which preprocessing techniques were used with justification of your decision.*

In [0]:
# Please comment your code
import numpy as np
import pprint as pp
import re
from bs4 import BeautifulSoup

# newlist has tokenized words from training data including duplicates (Corpus)
# list_unique_words has tokenized unique words in a list



#--------------------------------

#Assume that we have the following character instances
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',

            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z', '0']

# one-hot encoding and decoding 
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)


len_list = [len(s) for s in list_unique_words]
seq_length = max(len_list)

#Padding the words to make all words of the same size
def add_padding(corpus, seq_length):
    output = []
    for sentence in corpus:
        if len(sentence)>seq_length:
            output.append(sentence[:seq_length])
        else:
            for j in range(seq_length-len(sentence)):
                #sentence.append("<PAD>")
                sentence = sentence+"0"
            output.append(sentence)
    return output

#Padding the words to make all words of the same size
text_train_pad = add_padding(list_unique_words,seq_length )




In [0]:
# a list words for sequence data (input and output)
seq_data = text_train_pad

# Make a batch to have sequence data for input and ouput
def make_batch(seq_data):
    input_list = []
    input_batch = np.array(input_list)

    target_batch = []
    
    for seq in seq_data:
        # input data is:
        #     wor           woo        dee       div
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        
        input_data = [num_dic[n] for n in seq]

        
        # convert input to one-hot encoding.
        # if input is [3, 4, 4]:
        # [[ 0,  0,  0,  1,  0,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]]
        #input_batch.append(np.eye(dic_len)[input_data])
        input_batch = np.eye(dic_len)[input_data]

        target_batch.append(input_batch)
        #target_batch.append([target])

    return target_batch

input_batch2 = np.array(make_batch(seq_data))

target_batch2 = trained_embeddings_skipgram



### 2.2.2. Build Character Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
### Setting hyperparameters

learning_rate = 0.1
n_hidden = 30
total_epoch = 50

# Number of sequences for RNN
n_step = 3

# number of inputs (dimension of input vector) = 27
n_input = dic_len
# number of classes = 27

#n_class = dic_len
#n_class is the size of the word embedding
n_class = 51

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # LSTM layer, the batch_first is False by default, which means the input and output tensors are provided as (seq_len, batch_size, feature) 
        # We need to set it to True because we are using input of shape (batch_size, seq_len, feature)  
        # Apply dropout to prevent overfitting, you can try to change the dropout rate. Note that this dropout is applied on outputs of each LSTM layer except the last layer
        self.lstm = nn.LSTM(n_input, n_hidden, num_layers=2, batch_first =True, dropout=0.2)
        # Linear layer for output
        self.linear = nn.Linear(n_hidden,n_class)

    def forward(self, x):
        # There are two outputs from nn.LSTM:
        # 1. tensor of shape (batch_size, seq_len, hidden_size) containing the output features from the last layer of the LSTM for each time step t
        # 2. the tuple containing the hidden state and cell state.  
        # Here we only care about the first output. Details for the two outputs can be found in PyTorch documentation for nn.LSTM: https://pytorch.org/docs/stable/nn.html#lstm
        x,_ = self.lstm(x)
        # Here we extract only the last hidden state from the LSTM output features
        # The last hidden carries the information about what the LSTM cell has seen over the time. 
        # Thus the prediction based on the last hidden state not only considers the data at the current time step, instead, it considers historical data.
        x = self.linear(x[:,-1,:])
        x = F.log_softmax(x, dim=1)
        return x



### 2.1.4. Train Character Embeddings Model

In [0]:
# Please comment your code
import torch
#You can enable GPU here (cuda); or just CPU
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to GPU
#net = Net().to(device)
net = Net()

# Loss function and optimizer
#criterion = nn.NLLLoss()
criterion = nn.MSELoss()

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Preparing input
#input_batch = make_batch(seq_data)
#target_batch = trained_embeddings_skipgram

#test input
input_batch = make_batch(seq_data)
target_batch = trained_embeddings_skipgram



# Convert input into tensors and move them to GPU by uting tensor.to(device)
#input_batch_torch = torch.from_numpy(np.array(input_batch)).float().to(device)
#target_batch_torch = torch.from_numpy(np.array(target_batch)).view(-1).to(device)

input_batch_torch = torch.from_numpy(np.array(input_batch)).float()
target_batch_torch = torch.from_numpy(np.array(target_batch.T))


for epoch in range(total_epoch):  
    
    # Set the flag to training
    net.train()
    
    # forward + backward + optimize
    outputs = net(input_batch_torch) 
    loss = criterion(outputs, target_batch_torch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Set the flag to evaluation, which will 'turn off' the dropout
    net.eval()
    outputs = net(input_batch_torch) 
    
    # Evaluation loss and accuracy calculation
    loss = criterion(outputs, target_batch_torch)
    #_, predicted = torch.max(outputs, 1)
    #acc= accuracy_score(predicted.cpu().numpy(),target_batch_torch.cpu().numpy())

    print('Epoch: %d, loss: %.5f' %(epoch + 1, loss.item()))

print('Finished Training')



In [22]:
net.eval()
hidden_state = net(input_batch_torch)
print(n_hidden)
print(len(seq_data)) 
print(hidden_state.size())
print(hidden_state.data)

30
72732
torch.Size([72732, 51])
tensor([[-3.9320, -3.9322, -3.9521,  ..., -3.9422, -3.9473, -3.9254],
        [-3.9336, -3.9319, -3.9540,  ..., -3.9421, -3.9472, -3.9271],
        [-3.9332, -3.9319, -3.9536,  ..., -3.9420, -3.9473, -3.9268],
        ...,
        [-3.9323, -3.9320, -3.9529,  ..., -3.9418, -3.9476, -3.9266],
        [-3.9272, -3.9330, -3.9466,  ..., -3.9423, -3.9478, -3.9209],
        [-3.9392, -3.9316, -3.9584,  ..., -3.9434, -3.9457, -3.9303]])


### 2.1.5. Save Character Embeddings Model

In [0]:
# Please comment your code

### 2.1.6. Load Character Embeddings Model

In [0]:
# Please comment your code

## 2.3. Sequence model

### 2.3.1. Apply/Import Word Embedding and Character Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.3.2. Build Sequence Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.3.3. Train Sequence Model

In [0]:
# Please comment your code

### 2.3.4. Save Sequence Model

In [0]:
# Please comment your code

### 2.3.5. Load Sequence Model

In [0]:
# Please comment your code

# 3 - Evaluation

(*Please show your empirical evidence*)

## 3.1. Performance Evaluation


You are required to provide the table with precision, recall, f1 of test set.

In [0]:
# Please comment your code

## 3.2. Hyperparameter Testing
*You are required to draw a graph(y-axis: f1, x-axis: epoch) for test set and explain the optimal number of epochs based on the learning rate you have already chosen.*

In [0]:
# Please comment your code

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed* 

In [0]:
# If you used OOP style, use this section